In [22]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 24, Finished, Available, Finished)

In [23]:
from pyspark.sql import functions as f
df_dq_silver=spark.table("silver_transactions")

StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 25, Finished, Available, Finished)

In [24]:
df_dq_gold=spark.table("gold_fact_transactions")

StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 26, Finished, Available, Finished)

In [25]:
silver_count=df_dq_silver.count()
gold_count=df_dq_gold.count()

StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 27, Finished, Available, Finished)

In [26]:
if silver_count != gold_count:
   raise Exception (
        f"row count mismatch:silver={silver_count},gold={gold_count}"
    )

StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 28, Finished, Available, Finished)

In [27]:
silver_amount = (
    df_dq_silver
    .agg(f.sum("amount").alias("total_amount"))
    .collect()[0]["total_amount"]
)

gold_amount = (
    df_dq_gold
    .agg(f.sum("amount").alias("total_amount"))
    .collect()[0]["total_amount"]
)


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 29, Finished, Available, Finished)

In [28]:
if silver_amount != gold_amount:
    raise Exception(
        f"Amount mismatch: silver={silver_amount}, gold={gold_amount}"
    )


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 30, Finished, Available, Finished)

In [29]:
dq_nulls = df_dq_gold.filter(
    f.col("transaction_id").isNull() |
    f.col("customer_sk").isNull() |
    f.col("amount").isNull()
)


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 31, Finished, Available, Finished)

In [30]:
if dq_nulls.count() > 0:
    raise Exception("DQ failure: Nulls found in critical fact columns")


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 32, Finished, Available, Finished)

In [31]:
dup_txns = (
    df_dq_gold
    .groupBy("transaction_id")
    .count()
    .filter(f.col("count") > 1)
)


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 33, Finished, Available, Finished)

In [32]:
display(dup_txns)

StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f9eb03e2-84d2-4e76-a322-5e05606cf125)

In [34]:
audit_df = spark.createDataFrame(
    [(
        "gold_fact_pipeline",
        silver_count,
        gold_count,
        silver_amount,
        gold_amount,
        "SUCCESS"
    )],
    [
        "pipeline_name",
        "silver_count",
        "gold_count",
        "silver_amount",
        "gold_amount",
        "status"
    ]
)


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 36, Finished, Available, Finished)

In [35]:
audit_df.write.mode("append").saveAsTable("pipeline_audit_log")


StatementMeta(, a8eb448b-334a-4d57-a426-0b4856fe55be, 37, Finished, Available, Finished)